In [19]:
# Import essential libraries
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

from l2_pipeline import L2DatasetLoader, GPUDataExtractor, SelectiveInterpolator  # updated import to follow SOLID principles

In [20]:
! dir "E:\satdata"

 Volume in drive E is Data
 Volume Serial Number is 18AA-E419

 Directory of E:\satdata

04/15/2025  09:19 AM    <DIR>          .
04/15/2025  09:18 AM    <DIR>          Atchafalaya_2021-06-21_2021-07-21
04/15/2025  09:19 AM    <DIR>          Atchafalaya_2022-03-18_2022-04-16
04/15/2025  09:19 AM    <DIR>          Atchafalaya_2022-06-19_2022-07-19
04/15/2025  09:08 AM    <DIR>          Mississippi_2021-06-21_2021-07-21
04/15/2025  09:19 AM    <DIR>          Mississippi_2022-03-18_2022-04-16
04/15/2025  09:19 AM    <DIR>          Mississippi_2022-06-19_2022-07-19
               0 File(s)              0 bytes
               7 Dir(s)  1,728,219,213,824 bytes free


In [21]:
# Specify the data directory and search recursively for netCDF files
data_dir = r"E:\satdata\Mississippi_2021-06-21_2021-07-21"
nc_files = glob.glob(os.path.join(data_dir, '**', '*.nc'), recursive=True)

In [22]:
import xarray as xr

# Pick one file to inspect
file_path = nc_files[0]

geo = xr.open_dataset(file_path, group="geophysical_data")

print("\n🌊 GEOPHYSICAL VARIABLES:")
for var in geo.variables:
    print(f"{var}: {geo[var].dims} – {geo[var].shape}")



🌊 GEOPHYSICAL VARIABLES:
aot_869: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
angstrom: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_412: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_443: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_469: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_488: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_531: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_547: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_555: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_645: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_667: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Rrs_678: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
chlor_a: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
Kd_490: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
pic: ('number_of_lines', 'pixels_per_line') – (2030, 1354)
poc: ('number_of_lines', 'pixels_

In [ ]:
loader = L2DatasetLoader(variable='Rrs_645')
datasets = loader.load_multiple(nc_files)
print(f"Loaded {len(datasets)} datasets.")

Loaded 41 datasets.


In [ ]:
# Extract Rrs_645 and apply empirical formula
rrs_645 = ds['Rrs_645'].values

# Nechad et al. 2010 empirical model for TSS in mg/L
a, b = 289.29, 0.1686
tss = a * rrs_645 / (1 - rrs_645 / b)

# Mask NaNs or invalid values
tss = np.where(np.isnan(rrs_645) | (rrs_645 >= b), np.nan, tss)

# Plot it
plt.figure(figsize=(10, 6))
plt.imshow(tss, cmap='plasma', vmin=0, vmax=100)
plt.colorbar(label="TSS (mg/L)")
plt.title("Estimated Total Suspended Solids (TSS)")
plt.xlabel("Pixel")
plt.ylabel("Line")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'f:\\Programming\\GitHub\\nasa-murep-local\\river_plumes\\your_file.nc'